In [1]:
import stackprinter  # type: ignore
import jupyter_black  # type: ignore
from dotenv import load_dotenv  # type: ignore
import time

from baml_agents import init_logging, with_model
from baml_client.async_client import b

init_logging()
stackprinter.set_excepthook()
load_dotenv()
jupyter_black.load()

b = with_model(b, "gpt-4.1-nano")

# Using BAML in Jupyter notebokos

## 1. Streaming outputs

BAML offers a great developer experience, and I used some utilities to recreate some of the magic in Jupyter notebooks.

In [2]:
from baml_agents import JupyterOutputBox

with JupyterOutputBox(clear_after_finish=False) as output_box:
    s = ""
    for c in "abcdefghijklmnopqrstuvwxyz" * 3:
        s += c
        output_box.update(s)
        time.sleep(0.008)

# The letters are generated and displayed as they are being generated.

abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz

In [3]:
from baml_agents import JupyterBamlCollector

with JupyterOutputBox(clear_after_finish=False) as s:
    c = JupyterBamlCollector(b, stream_callback=s.display(formatter="json"))
    await c.b.InteractiveBamlJupyter_WriteShortStory()

{
 "genre": "Adventure",
 "characters": [
 {
 "name": "Lila",
 "age": 17,
 "occupation": "Explorer"
 },
 {
 "name": "Max",
 "age": 18,
 "occupation": "Guide"
 },
 {
 "name": "Sable",
 "age": 5,
 "occupation": "Pet fox"
 }
 ],
 "story_summary": "Lila and Max discover a hidden cave filled with ancient symbols. Sable prowls ahead, guiding them to treasure and secrets long buried beneath the mountain's shadow."
}

![Image](https://github.com/user-attachments/assets/ca21b62a-48b7-4a16-99fa-20bbe61f68b2)

## 2. Inspecting prompts and completions

In [4]:
await c.display_calls()

![Image](https://github.com/user-attachments/assets/59ed3cf3-84df-45a3-bf18-44dfbc457e4e)

You can also group prompts and completions by LLM call and by LLM call session:

In [5]:
# Generate some calls
c = JupyterBamlCollector(b)
await c.b.InteractiveBamlJupyter_WriteShortStory()
await c.b.InteractiveBamlJupyter_WriteShortStory()

# Display the calls as a single session
await c.display_session("Story generation session")

![Image](https://github.com/user-attachments/assets/7bed8f67-efe9-48c7-918a-5d0853dded95)

## 3. Convenience class

The `JupyterOutputBox` and `JupyterBamlCollector` are available as a single convenience class:

In [6]:
from baml_agents import JupyterBamlMonitor

with JupyterBamlMonitor(b) as m:
    # Output is displayed in real time in the notebook
    await m.b.InteractiveBamlJupyter_WriteShortStory()
    # Outout is hidden from the notebook

# Buttons with prompt and completino are displayed in the notebook
await m.display_calls()

{'genre': 'Fantasy',
 'characters': [{'name': 'Luna', 'age': 16, 'occupation': 'Apprentice mage'},
 {'name': 'Elder Thorne', 'age': 70, 'occupation': 'Village elder'},
 {'name': 'Finn', 'age': 18, 'occupation': 'Blacksmith apprentice'}],
 'story_summary': 'Luna discovers a hidden portal in the woods, leading to a mystical realm. With Elder Thorne’s guidance, she embarks on a quest to restore balance between worlds.'}

<IPython.core.display.Javascript object>

## 4. Interactive chat

Chat with your BAML Agent right in the notebook:

In [7]:
from baml_agents import ChatMessage, JupyterChatWidget


async def callback(chat_history):
    msg = ChatMessage(
        content=f"I'm seeing {len(chat_history)} messages",
        role="assistant",
    )
    return [msg]


JupyterChatWidget(callback).display()

Box(children=(HTML(value='Loading...', layout=Layout(display='none')), Textarea(value='', continuous_update=Fa…

![Image](https://github.com/user-attachments/assets/df46af62-97e5-42d6-8def-91762a46e432)

You can also pre-fill chat:

In [8]:
chat_history = [
    ChatMessage(
        content="Hey",
        role="user",
    ),
    ChatMessage(
        content="Hello",
        role="assistant",
    ),
]

JupyterChatWidget(callback, chat_history=chat_history).display()

Box(children=(HTML(value='Loading...', layout=Layout(display='none')), Textarea(value='', continuous_update=Fa…

You can also invoke the callback automatically:

In [9]:
chat = JupyterChatWidget(callback)
await chat.run("It's a beautiful day")
# Callback is being run automatically on startup to respond to the "It's a beautiful day" message

Box(children=(HTML(value='Loading...', layout=Layout(display='none')), Textarea(value='', continuous_update=Fa…